In [1]:
import cv2
import numpy as np
import math

In [170]:
img = cv2.imread("hough.jpg",0)

In [211]:
def Pad(img):
    Ix=list()
    for line in img:
        line = list(line)
        line.insert(0,0)        #Column padding to perform convolution
        line.insert(len(line),0)
        Ix.append(line)
    Opx = list()
    RowPad = [ 0 for i in range(len(Ix[0])) ] #Row padding to perform convolution
    Opx.append(RowPad)
    for i in Ix:
        Opx.append(i)
    Opx.append(RowPad)    
    Opx = np.asarray(Opx)
    return Opx

In [4]:
N = img.shape[0]
M = img.shape[1]
Img_padx = Pad(img)  #Pad Image

In [5]:
nx = [[1,0,-1],
      [2,0,-2],
      [1,0,-1]]  #Sobel X kernel

for i in nx:
        temp = i[0]
        i[0] = i[2]
        i[2] = temp

In [56]:
sobelxImage = [[0]*M for i in range(N)]
sobelxImage = np.asarray(sobelxImage)
sobelyImage = [[0]*M for i in range(N)]
sobelyImage = np.asarray(sobelyImage)
    
imagex = Img_padx
for i in range(1,N):     # Loop over every pixel of the image
    for j in range(1,M):
        x = imagex[i-1:i+2,j-1:j+2]
        result = [[a*b for a, b in zip(k, l)] for k, l in zip(x, nx)] #Element wise Multiplication
        elex = 0
        for row in result:
            for ele in row: 
                elex = elex+ele  #Addition of the products to obtain value for that pixel
        
            sobelxImage[i-1][j-1] = elex    

In [57]:
cv2.imwrite("ConvX_image.png",sobelxImage)

True

In [209]:
def find_edges(img):  #Find the edges
    eximg = np.zeros((img.shape[0],img.shape[1]))
    for i in range(N):
        for j in range(M):
            if img[i,j] > 100:
                eximg[i,j] = 255
    return eximg            

In [210]:
edx = find_edges(sobelxImage)
cv2.imwrite("edx.jpg",edx)

True

In [61]:
def find_rho(theta,x,y): #Find rho values
    return int(round(x*math.cos(math.radians(theta))+y*math.sin(math.radians(theta))))+diaglen

In [62]:
diaglen = int(math.sqrt((N*N) + (M*M))) 
accu = np.zeros((179,(2*diaglen)-1)) #Accumulator Matrix

In [63]:
def cal_pixrho(x,y):  #Taking angles from -89 to 90 for a pixel value
    rhoval = []
    for i in range(-89,90):
        rhoval.append(find_rho(i,x,y))
    return rhoval    

In [64]:
for i in range(N):   #Building Accumulator Matrix
    for j in range(M):
        if edx[i,j] == 255:
            pixrho = cal_pixrho(i,j)
            for k in range(179):
                accu[k][pixrho[k]] = accu[k][pixrho[k]] + 1 
        

In [91]:
def max_indices(arr, k):   #Obtaining all the peak occurences
    assert k <= arr.size, 
    arr_ = arr.astype(float)  
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return np.asarray(max_idxs)

In [181]:
k = []
for i in range(lp.shape[0]):
    n = lp[i][0].shape[0]
    if n>1:
        for j in range(n):
            k.append([lp[i][0][j], lp[i][1][j]])
    else:
        k.append([lp[i][0][0], lp[i][1][0]])

In [182]:
lp = max_indices(accu, 100)

In [183]:
imgv = cv2.imread("hough.jpg")      #Plotting vertical and horizontal lines
imgd = cv2.imread("hough.jpg")
for t,r in k:
    for i in range(N):
        for j in range(M):
            if edx[i,j] == 255:
                brho = find_rho(t-89,i,j)
                if brho == r:
                    if t >= 0 and t<=5:
                        imgv[i,j] = (0,255,0)
                    elif t >=33 and t<=37:
                        imgd[i,j] = (0,255,0)
        

In [180]:
cv2.imwrite("red_line.jpg",imgv)
cv2.imwrite("blue_line.jpg",imgd)

True

In [212]:
cv2.imwrite("accu.jpg",accu)

True